Once we have extracted the edits that have happened to a neuron (see [here](./l2_edit_extraction.ipynb)), it can be helpful to
replay them in order to see how the neuron has changed over time.


## Extract the edits and initial state of this neuron


In [1]:
import networkx as nx
from tqdm.auto import tqdm

from caveclient import CAVEclient
from paleo import get_initial_graph, get_root_level2_edits

/Users/ben.pedigo/code/cave-edits/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


As in the previous example, we'll start by extracting the edits to a neuron.


In [2]:
root_id = 864691135639556411

client = CAVEclient("minnie65_public", version=1078)

networkdeltas = get_root_level2_edits(root_id, client)

Extracting level2 edits: 100%|██████████| 693/693 [01:27<00:00,  7.90it/s]


This time, we'll also use `paleo.get_initial_graph` to get the level2 graph connectivity
for all objects that participate in this neuron's edit history. This will allow us to
replay the edits in the context of the full segmentation graph.


In [3]:
graph = get_initial_graph(root_id, client)

/Users/ben.pedigo/code/cave-edits/.venv/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
100%|██████████| 309/309 [00:30<00:00, 10.09it/s]


## Replaying the edits over the level2 graph


The simplest thing we can do now is to replay the edits in order. `paleo` provides the
`apply_edit` function that takes in the graph and an edit and applies it to the graph.
Note that this modifies the graph in place.


In [4]:
from paleo import apply_edit

deltas = list(networkdeltas.values())

for delta in tqdm(deltas, disable=False):
    apply_edit(graph, delta)

100%|██████████| 693/693 [00:00<00:00, 47637.55it/s]


As a sanity check, we might want to compare the graph that we got from replaying edits
from the original, to the actual graph that we'd get from `client.chunkedgraph.level2_chunk_graph`.

To do so, we need to also know a point on the object of interest to use as an anchor point -
this is because typically `graph` will be composed of many connected components, but only
one of them corresponds to the current state of our neuron.


In [5]:
from paleo import get_nucleus_supervoxel

nuc_supervoxel_id = get_nucleus_supervoxel(root_id, client)

nuc_level2_id = client.chunkedgraph.get_roots(nuc_supervoxel_id, stop_layer=2)[0]

neuron_component = nx.node_connected_component(graph, nuc_level2_id)
neuron_graph = graph.subgraph(neuron_component)

In [6]:
computed_edgelist = nx.to_pandas_edgelist(neuron_graph).values.astype(int)

In [7]:
final_edgelist = client.chunkedgraph.level2_chunk_graph(root_id)

It's assuring to see that we at least have the same number of edges in both cases.


In [8]:
len(final_edgelist), len(computed_edgelist)

(10210, 10210)

...and when we compare the actual edgelists element-wise, we see that they are the same.


In [9]:
import numpy as np

final_edgelist = np.unique(np.sort(final_edgelist, axis=1), axis=0)
computed_edgelist = np.unique(np.sort(computed_edgelist, axis=1), axis=0)

(final_edgelist == computed_edgelist).all()

True

## Tracking neuron state over the edit history


Now, let's try keeping track of the state of the neuron at every point along this edit
history.

This becomes just a bit more complicated: often the level2 ID corresponding to
a nucleus's location may change over time if there was an edit near that location. If
we want to keep track of the segmentation component corresponding to the nucleus
(or some other point) over this whole history,
then we need to know how this ID changes over time. `paleo` provides the
`get_node_aliases` function to help with this.


In [10]:
from paleo import get_node_aliases

node_info = get_node_aliases(nuc_supervoxel_id, client, stop_layer=2)
node_info

,start_valid_ts,end_valid_ts
node_id,,
161513998385152439,2020-08-01 13:07:22.739000+00:00,2024-06-05 10:10:01.203215+00:00
161513998385152001,2020-05-29 13:26:43.761000+00:00,2020-08-01 13:07:22.738999+00:00


Now we have all the ingredients to replay the edits and keep track of the neuron's state.


In [11]:
def find_level2_node(graph, level2_ids):
    for level2_id in level2_ids:
        if graph.has_node(level2_id):
            return level2_id
    return None


# keep track of components that are reached as we go
components = []

# store the initial state
nucleus_node_id = find_level2_node(graph, node_info.index)
component = nx.node_connected_component(graph, nucleus_node_id)
components.append(component)

# after each edit, apply it and store the connected component for the nucleus node
for delta in tqdm(deltas, disable=False):
    apply_edit(graph, delta)
    nucleus_node_id = find_level2_node(graph, node_info.index)
    component = nx.node_connected_component(graph, nucleus_node_id)
    components.append(component)

100%|██████████| 693/693 [00:02<00:00, 272.41it/s]


In [12]:
from paleo import get_component_masks

l2_masks = get_component_masks(components)
l2_masks

,0,1,2,3,4,5,6,7,8,9,...,684,685,686,687,688,689,690,691,692,693
150388177863966928,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
150458546608144530,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
150528846632845407,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
150528846632845424,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
150528915352323074,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181993776573580123,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
181993845293056671,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
181993845293057025,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
182064145317757549,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [18]:
persistent_mask = l2_masks.sum(axis=1) == l2_masks.shape[1]
persistent_mask.sum()

8962

## TODO: describe a function for wrapping up some of this boilerplate


In [14]:
def apply_edits(graph, deltas, anchor_supervoxel_id, client):
    if anchor_supervoxel_id in ["nucleus", "soma"]:
        anchor_supervoxel_id = get_nucleus_supervoxel(root_id, client)

    for delta in tqdm(deltas, disable=False):
        apply_edit(graph, delta)

    return graph